## Importing necessary libraries

In [2]:
import PyPDF2
import pandas as pd
import tabula
import os

## Table 1 - Statement of Net Position

### Finding page numbers

In [36]:
def find_keywords_in_pdf(keywords, pdf_file):
    pg=[]
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        # Loop through all pages
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text().lower()
            
            # Check if all keywords exist on this page
            if all(keyword.lower() in text for keyword in keywords):
                if page_num>=30:
                    pg.append(page_num+1)
                    break
                #print(f"Keywords '{keywords}' found on page {page_num + 1}")
    return pg[0] if pg else None

### Scraping tables

In [41]:
def extract_tables_from_pdf(pdf_file, start_page, num_pages=2):
    # Extract tables from the specified range of pages
    end_page = start_page + num_pages - 1
    tables = tabula.read_pdf(pdf_file, pages=f"{start_page}-{end_page}", multiple_tables=True)
    
    return tables

### Looping through each pdf

In [42]:
pdf_file = ['C:/Users/sarah/Downloads/OR2023ACFR.pdf','C:/Users/sarah/Downloads/SC2023ACFR.pdf', 'C:/Users/sarah/Downloads/MI2023ACFR.pdf', 'C:/Users/sarah/Downloads/KS2023ACFR.pdf', 'C:/Users/sarah/Downloads/NJ2023ACFR.pdf']
tables_dict = {}
for pdf in pdf_file:
    first_page_number = find_keywords_in_pdf(
        ["STATEMENT OF NET POSITION", "Primary Government", "Governmental", "Business-type", "Component"], 
        pdf
    )
    tables=(extract_tables_from_pdf(pdf,first_page_number))
    df_list = []
    for i, table in enumerate(tables):
        df = pd.DataFrame(table)
        df_list.append(df)
    tables_dict[pdf] = df_list
    print(first_page_number)


39
38
35


unknown widths : 
[0, IndirectObject(1752, 0, 2159789394352)]
unknown widths : 
[0, IndirectObject(1756, 0, 2159789394352)]
unknown widths : 
[0, IndirectObject(1760, 0, 2159789394352)]
unknown widths : 
[0, IndirectObject(1764, 0, 2159789394352)]
unknown widths : 
[0, IndirectObject(1768, 0, 2159789394352)]
unknown widths : 
[0, IndirectObject(1772, 0, 2159789394352)]


35
42


In [40]:
for key, value in tables_dict.items():
    print(f"{key} has {(value)} items.")

C:/Users/sarah/Downloads/OR2023ACFR.pdf has [      Capital Assets (net)  17,385,072     635,019  18,020,091   7,708,411
0  Total Noncurrent Assets  24,841,150   3,059,997  27,901,147  15,692,991
1             Total Assets  48,954,089  10,745,046  59,699,135  23,286,027,                                      Unnamed: 0 Unnamed: 1 Governmental  \
0                                           NaN        NaN   Activities   
1                                          SETS        NaN          NaN   
2                                 rrent Assets:        NaN          NaN   
3                     Cash and Cash Equivalents          $   15,383,971   
4        Cash and Cash Equivalents - Restricted        NaN    2,731,702   
5                                   Investments        NaN    1,499,544   
6                      Investments - Restricted        NaN      484,119   
7                 Securities Lending Collateral        NaN       11,634   
8        Accounts and Interest Receivable (net)       

In [43]:
# Define a function to rename columns based on index
def rename_columns(df):
    # Define a dictionary mapping the index of the column to the desired name
    rename_map = {
        0: 'Header',
        1: 'Unnamed0',
        2: 'Governmental Activities',
        3: 'Unnamed2',
        4: 'Business-type Activities',         # 3rd column
        6: 'Total',  # 5th column
        8: 'Component Units',    # 7th column                        
    }
    
    # Get the current number of columns
    num_columns = len(df.columns)
    
    # Loop through the rename_map and rename only if the column index exists in the DataFrame
    for index, new_name in rename_map.items():
        if index < num_columns:
            df.columns.values[index] = new_name

In [44]:
# Define custom column names for the first and second DataFrames

second_df_columns = ['Header','Governmental Activities', 'Unnamed2','Business-Type Activities','Unnamed4', 'Total','Unnamed6', 'Component Units']  # List format for easier column insertion

# Iterate through the DataFrames in tables_dict and rename/add columns
for key in tables_dict.keys():
    print(f"\nRenaming columns for DataFrames under: {key}")
    
    # Iterate through the DataFrames for the current key
    for i, df in enumerate(tables_dict[key]):
        print(f"\nBefore renaming: DataFrame {i + 1} for {key}")
        print("Columns before renaming:", df.columns)

        if i == 0:  # First DataFrame
            # Rename columns for the first DataFrame based on the first_df_columns mapping
            rename_columns(df)

        elif i == 1:  # Second DataFrame
            # Add header above the first row explicitly for the second DataFrame
            new_header = pd.DataFrame([second_df_columns], columns=range(len(second_df_columns)))
            
            # Concatenate new header and the rest of the original DataFrame (resetting index so the new row is added correctly)
            df.reset_index(drop=True, inplace=True)
            df = pd.concat([new_header, df], ignore_index=True)

        # You can add more conditions here if you want to rename subsequent DataFrames differently.

        # Update the modified DataFrame back into the dictionary
        tables_dict[key][i] = df

        print(f"\nAfter renaming or adding header: DataFrame {i + 1}")
        # Print the DataFrame to check the result
        print("after:", df.columns)



Renaming columns for DataFrames under: C:/Users/sarah/Downloads/OR2023ACFR.pdf

Before renaming: DataFrame 1 for C:/Users/sarah/Downloads/OR2023ACFR.pdf
Columns before renaming: Index(['Capital Assets (net)', '17,385,072', '635,019', '18,020,091',
       '7,708,411'],
      dtype='object')

After renaming or adding header: DataFrame 1
after: Index(['Assets', '17,385,072', 'Governmental Activities', '18,020,091',
       'Business-type Activities'],
      dtype='object')

Before renaming: DataFrame 2 for C:/Users/sarah/Downloads/OR2023ACFR.pdf
Columns before renaming: Index(['Unnamed: 0', 'Unnamed: 1', 'Governmental', 'Unnamed: 2',
       'Business-type', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Component'],
      dtype='object')

After renaming or adding header: DataFrame 2
after: Index([              0,               1,               2,               3,
                     4,               5,               6,               7,
          'Unnamed: 0',    'Unnamed: 1',  'Governmental'

In [ ]:
import pandas as pd
import os
import re

# Define the directory where the Excel files are saved
output_dir = 'C:/Users/sarah/Downloads/ouput'

# Keywords for searching and corresponding column names
keyword_dict = {
    'Unrestricted': ['ga_un_asts', 'bta__un_asts', '','ga_totalexp'],
    'Current assets': ['bta__un_asts', 'bta_grants', 'bta_capitalgrants','bta_totalexp'],
    'Current Liabilities': ['comp_un_asts', 'comp_grants', 'comp_capitalgrants','comp_totalexp']
}

# Initialize a list to store the found values with their respective filenames
found_data = []

# Loop through each Excel file in the output directory
for file_name in os.listdir(output_dir):
    if file_name.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(output_dir, file_name)

        try:
            # Read the Excel file into a DataFrame, specifying the engine
            df = pd.read_excel(file_path, engine='openpyxl')

            # Lowercase the relevant column for better matching
            df['Functions/Programs'] = df['Functions/Programs'].str.lower()

            # Initialize a dictionary for storing the values for this file
            found_values = {'file_name': file_name}
            for keys in keyword_dict.values():
                for key in keys:
                    found_values[key] = None  # Initialize each column as None

            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Charges for Services' in df.columns:
                for keyword, keys in keyword_dict.items():
                    # Use regex for flexible matching
                    row_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Charges for Services']
                        found_values[keys[0]] = value  # Assign value to 'ga_chgs', 'bta_chgs', or 'comp_chgs'

            if 'Functions/Programs' in df.columns and 'Operating Grants and Contributions' in df.columns:
                        # Attempt to find "Operating Grants and Contributions"
                    op_grant_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if op_grant_mask.any():
                        op_grant_index = df[op_grant_mask].index[0]
                        op_grant_value = df.loc[op_grant_index, 'Operating Grants and Contributions']
                        found_values[keys[1]] = op_grant_value  # Assign value to 'ga_grants', 'bta_grants', or 'comp_grants'

            if 'Functions/Programs' in df.columns and 'Capital Grants and Contributions' in df.columns:           
                        # Attempt to find "Capital Grants and Contributions"
                    cap_grant_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if cap_grant_mask.any():
                        cap_grant_index = df[cap_grant_mask].index[0]
                        cap_grant_value = df.loc[cap_grant_index, 'Capital Grants and Contributions']
                        found_values[keys[2]] = cap_grant_value  # Assign value to 'ga_capitalgrants', 'bta_capitalgrants', or 'comp_capitalgrants'
                        
            if 'Functions/Programs' in df.columns and 'Expenses' in df.columns:               
                    expenses_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if expenses_mask.any():
                        expenses_index = df[expenses_mask].index[0]
                        expenses_value = df.loc[expenses_index, 'Expenses']
                        found_values[keys[3]] = expenses_value

            # Append the found values for this file (including file name) to the list
            found_data.append(found_values)

        except ValueError as ve:
            print(f"ValueError for file {file_name}: {ve}")
        except Exception as e:
            print(f"An error occurred for file {file_name}: {e}")

# Create a new DataFrame from the found data
found_df = pd.DataFrame(found_data)

# Save the new DataFrame to an Excel file
output_file_path = os.path.join(output_dir, 'found_charges_for_services_with_all_columns.xlsx')
found_df.to_excel(output_file_path, index=False)

print(f"Found values saved to {output_file_path}")


## Table 2 - Statement of Activities

### Finding page numbers

In [6]:
def find_keywords_in_pdf(keywords, pdf_file):
    pg=[]
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        # Loop through all pages
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text().lower()
            
            # Check if both keywords exist on this page
            if all(keyword.lower() in text for keyword in keywords):
                if page_num>=32:
                    pg.append(page_num+1)
                    break
                #print(f"Keywords '{keywords}' found on page {page_num + 1}")
    return pg[0] if pg else None

### Looping through each pdf

In [7]:
pdf_file = ['C:/Users/sarah/Downloads/OR2023ACFR.pdf','C:/Users/sarah/Downloads/SC2023ACFR.pdf', 'C:/Users/sarah/Downloads/MI2023ACFR.pdf', 'C:/Users/sarah/Downloads/KS2023ACFR.pdf', 'C:/Users/sarah/Downloads/NJ2023ACFR.pdf']
tables_dict = {}
for pdf in pdf_file:
    first_page_number = find_keywords_in_pdf(
        ["Statement of Activities", "Expenses", "Operating", "Capital"], 
        pdf
    )
    tables=extract_tables_from_pdf(pdf,first_page_number)
    df_list = []
    for i, table in enumerate(tables):
        df = pd.DataFrame(table)
        df_list.append(df)
    tables_dict[pdf] = df_list
    print(first_page_number)

41
40
37


unknown widths : 
[0, IndirectObject(1752, 0, 2159825636000)]
unknown widths : 
[0, IndirectObject(1756, 0, 2159825636000)]
unknown widths : 
[0, IndirectObject(1760, 0, 2159825636000)]
unknown widths : 
[0, IndirectObject(1764, 0, 2159825636000)]
unknown widths : 
[0, IndirectObject(1768, 0, 2159825636000)]
unknown widths : 
[0, IndirectObject(1772, 0, 2159825636000)]


48
34


In [8]:
for key, value in tables_dict.items():
    print(f"{key} has {(value)} items.")
    
        

C:/Users/sarah/Downloads/OR2023ACFR.pdf has [                    Functions/Programs Unnamed: 0  Unnamed: 1 Unnamed: 2  \
0                  Primary Government:        NaN         NaN        NaN   
1             Governmental Activities:        NaN         NaN        NaN   
2                            Education          $   9,102,586          $   
3                       Human Services        NaN  22,495,019        NaN   
4                        Public Safety        NaN   2,368,376        NaN   
5   Economic and Community Development        NaN   1,240,288        NaN   
6                    Natural Resources        NaN   1,115,340        NaN   
7                       Transportation        NaN   2,173,145        NaN   
8       Consumer and Business Services        NaN    5 20,636        NaN   
9                       Administration        NaN    9 67,392        NaN   
10                         Legislative        NaN     7 6,896        NaN   
11                            Judicial     

In [9]:
# Define a function to rename columns based on index
def rename_columns(df):
    # Define a dictionary mapping the index of the column to the desired name
    rename_map = {
        0: 'Functions/Programs',
        2: 'Expenses',
        4: 'Charges for Services',         # 3rd column
        6: 'Operating Grants and Contributions',  # 5th column
        8: 'Capital Grants and Contributions',    # 7th column
        10: 'Net'                           # 9th column (only if there are 9 or more columns)
    }
    
    # Get the current number of columns
    num_columns = len(df.columns)
    
    # Loop through the rename_map and rename only if the column index exists in the DataFrame
    for index, new_name in rename_map.items():
        if index < num_columns:
            df.columns.values[index] = new_name

In [10]:
# Define custom column names for the first and second DataFrames

second_df_columns = ['Header','Unnamed0','Governmental Activities', 'Unnamed2','Business-Type Activities','Unnamed4', 'Total','Unnamed6', 'Component Units']  # List format for easier column insertion

# Iterate through the DataFrames in tables_dict and rename/add columns
for key in tables_dict.keys():
    print(f"\nRenaming columns for DataFrames under: {key}")
    
    # Iterate through the DataFrames for the current key
    for i, df in enumerate(tables_dict[key]):
        print(f"\nBefore renaming: DataFrame {i + 1} for {key}")
        print("Columns before renaming:", df.columns)

        if i == 0:  # First DataFrame
            # Rename columns for the first DataFrame based on the first_df_columns mapping
            rename_columns(df)

        elif i == 1:  # Second DataFrame
            # Add header above the first row explicitly for the second DataFrame
            new_header = pd.DataFrame([second_df_columns], columns=range(len(second_df_columns)))
            
            # Concatenate new header and the rest of the original DataFrame (resetting index so the new row is added correctly)
            df.reset_index(drop=True, inplace=True)
            df = pd.concat([new_header, df], ignore_index=True)

        # You can add more conditions here if you want to rename subsequent DataFrames differently.

        # Update the modified DataFrame back into the dictionary
        tables_dict[key][i] = df

        print(f"\nAfter renaming or adding header: DataFrame {i + 1}")
        # Print the DataFrame to check the result
        print("after:", df.columns)



Renaming columns for DataFrames under: C:/Users/sarah/Downloads/OR2023ACFR.pdf

Before renaming: DataFrame 1 for C:/Users/sarah/Downloads/OR2023ACFR.pdf
Columns before renaming: Index(['Functions/Programs', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')

After renaming or adding header: DataFrame 1
after: Index(['Functions/Programs', 'Unnamed: 0', 'Expenses', 'Unnamed: 2',
       'Charges for Services', 'Unnamed: 4',
       'Operating Grants and Contributions', 'Unnamed: 6',
       'Capital Grants and Contributions', 'Unnamed: 8', 'Net'],
      dtype='object')

Before renaming: DataFrame 2 for C:/Users/sarah/Downloads/OR2023ACFR.pdf
Columns before renaming: Index(['Personal Income Taxes', 'Unnamed: 0', '13,641,111', 'Unnamed: 1', '-',
       'Unnamed: 2', '13,641,111.1', 'Unnamed: 3', '-.1'],
      dtype='object')

After renaming or adding header: DataFrame

In [11]:
import pandas as pd

# Define the output directory
output_dir = 'C:/Users/sarah/Downloads/'

# Iterate through tables_dict and save each DataFrame to an Excel file
for key in tables_dict.keys():
    for i, df in enumerate(tables_dict[key]):
        # Create a unique file name for each DataFrame
        output_file_path = f"{output_dir}{key.split('/')[-1].replace('.pdf', '')}_table_{i + 1}.xlsx"
        
        # Save the DataFrame to an Excel file
        df.to_excel(output_file_path, index=False)
        
        print(f"Saved DataFrame {i + 1} for {key} to {output_file_path}")


Saved DataFrame 1 for C:/Users/sarah/Downloads/OR2023ACFR.pdf to C:/Users/sarah/Downloads/OR2023ACFR_table_1.xlsx
Saved DataFrame 2 for C:/Users/sarah/Downloads/OR2023ACFR.pdf to C:/Users/sarah/Downloads/OR2023ACFR_table_2.xlsx
Saved DataFrame 1 for C:/Users/sarah/Downloads/SC2023ACFR.pdf to C:/Users/sarah/Downloads/SC2023ACFR_table_1.xlsx
Saved DataFrame 2 for C:/Users/sarah/Downloads/SC2023ACFR.pdf to C:/Users/sarah/Downloads/SC2023ACFR_table_2.xlsx
Saved DataFrame 1 for C:/Users/sarah/Downloads/MI2023ACFR.pdf to C:/Users/sarah/Downloads/MI2023ACFR_table_1.xlsx
Saved DataFrame 2 for C:/Users/sarah/Downloads/MI2023ACFR.pdf to C:/Users/sarah/Downloads/MI2023ACFR_table_2.xlsx
Saved DataFrame 3 for C:/Users/sarah/Downloads/MI2023ACFR.pdf to C:/Users/sarah/Downloads/MI2023ACFR_table_3.xlsx
Saved DataFrame 1 for C:/Users/sarah/Downloads/KS2023ACFR.pdf to C:/Users/sarah/Downloads/KS2023ACFR_table_1.xlsx
Saved DataFrame 2 for C:/Users/sarah/Downloads/KS2023ACFR.pdf to C:/Users/sarah/Download

In [30]:
import pandas as pd
import os
import re

# Define the directory where the Excel files are saved
output_dir = 'C:/Users/sarah/Downloads/ouput'

# Keywords for searching and corresponding column names
keyword_dict = {
    'Total Government Activities': ['ga_chgs', 'ga_grants', 'ga_capitalgrants','ga_totalexp'],
    'Total Business-Type Activities': ['bta_chgs', 'bta_grants', 'bta_capitalgrants','bta_totalexp'],
    'Total Component Units': ['comp_chgs', 'comp_grants', 'comp_capitalgrants','comp_totalexp']
}

# Initialize a list to store the found values with their respective filenames
found_data = []

# Loop through each Excel file in the output directory
for file_name in os.listdir(output_dir):
    if file_name.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(output_dir, file_name)

        try:
            # Read the Excel file into a DataFrame, specifying the engine
            df = pd.read_excel(file_path, engine='openpyxl')

            # Lowercase the relevant column for better matching
            df['Functions/Programs'] = df['Functions/Programs'].str.lower()

            # Initialize a dictionary for storing the values for this file
            found_values = {'file_name': file_name}
            for keys in keyword_dict.values():
                for key in keys:
                    found_values[key] = None  # Initialize each column as None

            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Charges for Services' in df.columns:
                for keyword, keys in keyword_dict.items():
                    # Use regex for flexible matching
                    row_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Charges for Services']
                        found_values[keys[0]] = value  # Assign value to 'ga_chgs', 'bta_chgs', or 'comp_chgs'

            if 'Functions/Programs' in df.columns and 'Operating Grants and Contributions' in df.columns:
                        # Attempt to find "Operating Grants and Contributions"
                    op_grant_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if op_grant_mask.any():
                        op_grant_index = df[op_grant_mask].index[0]
                        op_grant_value = df.loc[op_grant_index, 'Operating Grants and Contributions']
                        found_values[keys[1]] = op_grant_value  # Assign value to 'ga_grants', 'bta_grants', or 'comp_grants'

            if 'Functions/Programs' in df.columns and 'Capital Grants and Contributions' in df.columns:           
                        # Attempt to find "Capital Grants and Contributions"
                    cap_grant_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if cap_grant_mask.any():
                        cap_grant_index = df[cap_grant_mask].index[0]
                        cap_grant_value = df.loc[cap_grant_index, 'Capital Grants and Contributions']
                        found_values[keys[2]] = cap_grant_value  # Assign value to 'ga_capitalgrants', 'bta_capitalgrants', or 'comp_capitalgrants'
                        
            if 'Functions/Programs' in df.columns and 'Expenses' in df.columns:               
                    expenses_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if expenses_mask.any():
                        expenses_index = df[expenses_mask].index[0]
                        expenses_value = df.loc[expenses_index, 'Expenses']
                        found_values[keys[3]] = expenses_value

            # Append the found values for this file (including file name) to the list
            found_data.append(found_values)

        except ValueError as ve:
            print(f"ValueError for file {file_name}: {ve}")
        except Exception as e:
            print(f"An error occurred for file {file_name}: {e}")

# Create a new DataFrame from the found data
found_df = pd.DataFrame(found_data)

# Save the new DataFrame to an Excel file
output_file_path = os.path.join(output_dir, 'found_charges_for_services_with_all_columns.xlsx')
found_df.to_excel(output_file_path, index=False)

print(f"Found values saved to {output_file_path}")


An error occurred for file KS2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file MI2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file MI2023ACFR_table_3.xlsx: 'Functions/Programs'
An error occurred for file NJ2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file NJ2023ACFR_table_3.xlsx: 'Functions/Programs'
An error occurred for file OR2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file SC2023ACFR_table_2.xlsx: 'Functions/Programs'
Found values saved to C:/Users/sarah/Downloads/ouput\found_charges_for_services_with_all_columns.xlsx


In [25]:
import pandas as pd
import os
import re

# Define the directory where the Excel files are saved
output_dir = 'C:/Users/sarah/Downloads/ouput'

# Keywords for searching and corresponding column names
keyword_dict = {
    'Total Government Activities': 'ga_chgs',
    'Total Business-Type Activities': 'bta_chgs',
    'Total Component Units': 'comp_chgs'
}

# Initialize a list to store the found values with their respective filenames
found_data = []

# Loop through each Excel file in the output directory
for file_name in os.listdir(output_dir):
    if file_name.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(output_dir, file_name)

        try:
            # Read the Excel file into a DataFrame, specifying the engine
            df = pd.read_excel(file_path, engine='openpyxl')

            # Lowercase the relevant column for better matching
            df['Functions/Programs'] = df['Functions/Programs'].str.lower()

            # Initialize a dictionary for storing the values for this file
            found_values = {'file_name': file_name}
            for key in keyword_dict.values():
                found_values[key] = None  # Initialize each column as None

            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Charges for Services' in df.columns:
                for keyword, new_key in keyword_dict.items():
                    # Use regex for flexible matching
                    row_mask = df['Functions/Programs'].str.contains(re.escape(keyword.lower()), na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Charges for Services']
                        found_values[new_key] = value

            # Append the found values for this file (including file name) to the list
            found_data.append(found_values)

        except ValueError as ve:
            print(f"ValueError for file {file_name}: {ve}")
        except Exception as e:
            print(f"An error occurred for file {file_name}: {e}")

# Create a new DataFrame from the found data
found_df = pd.DataFrame(found_data)

# Save the new DataFrame to an Excel file
output_file_path = os.path.join(output_dir, 'found_charges_for_services_with_flexible_matching.xlsx')
found_df.to_excel(output_file_path, index=False)

print(f"Found values saved to {output_file_path}")


An error occurred for file KS2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file MI2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file MI2023ACFR_table_3.xlsx: 'Functions/Programs'
An error occurred for file NJ2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file NJ2023ACFR_table_3.xlsx: 'Functions/Programs'
An error occurred for file OR2023ACFR_table_2.xlsx: 'Functions/Programs'
An error occurred for file SC2023ACFR_table_2.xlsx: 'Functions/Programs'
Found values saved to C:/Users/sarah/Downloads/ouput\found_charges_for_services_with_flexible_matching.xlsx


In [23]:
import pandas as pd
import os

# Define the directory where the Excel files are saved
output_dir = 'C:/Users/sarah/Downloads/ouput'

# Keywords for searching and corresponding column names for Charges for Services
charges_keyword_dict = {
    'Total Government Activities': 'ga_chgs',
    'Total Business-Type Activities': 'bta_chgs',
    'Total Component Units': 'comp_chgs'
}

# Keywords for searching and corresponding column names for Operating Grants and Contributions
grants_keyword_dict = {
    'Total Government Activities': 'ga_grants',
    'Total Business-Type Activities': 'bta_grants',
    'Total Component Units': 'comp_grants'
}

# Keywords for searching and corresponding column names for Capital Grants and Contributions
capital_grants_keyword_dict = {
    'Total Government Activities': 'ga_capitalgrants',
    'Total Business-Type Activities': 'bta_capitalgrants',
    'Total Component Units': 'comp_capitalgrants'
}

# Initialize a list to store the found values with their respective filenames
found_data = []

# Loop through each Excel file in the output directory
for file_name in os.listdir(output_dir):
    if file_name.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(output_dir, file_name)

        try:
            # Read the Excel file into a DataFrame, specifying the engine
            df = pd.read_excel(file_path, engine='openpyxl')

            # Initialize a dictionary for storing the values for this file, with 'file_name'
            found_values = {'file_name': file_name}
            for key in charges_keyword_dict.values():
                found_values[key] = None  # Initialize each Charges for Services column as None
            for key in grants_keyword_dict.values():
                found_values[key] = None  # Initialize each Operating Grants for Contributions column as None
            for key in capital_grants_keyword_dict.values():
                found_values[key] = None  # Initialize each Capital Grants for Contributions column as None

            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Charges for Services' in df.columns:
                # Extract values for Charges for Services
                for keyword, new_key in charges_keyword_dict.items():
                    # Attempt to find the row that contains the keyword
                    row_mask = df['Functions/Programs'].str.contains(keyword, case=False, na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Charges for Services']
                        found_values[new_key] = value

            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Operating Grants and Contributions' in df.columns:
                # Extract values for Operating Grants and Contributions
                for keyword, new_key in grants_keyword_dict.items():
                    # Attempt to find the row that contains the keyword
                    row_mask = df['Functions/Programs'].str.contains(keyword, case=False, na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Operating Grants and Contributions']
                        found_values[new_key] = value
            
            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Capital Grants and Contributions' in df.columns:
                # Extract values for Capital Grants and Contributions
                for keyword, new_key in capital_grants_keyword_dict.items():
                    # Attempt to find the row that contains the keyword
                    row_mask = df['Functions/Programs'].str.contains(keyword, case=False, na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Capital Grants and Contributions']
                        found_values[new_key] = value

            # Append the found values for this file (including file name) to the list
            found_data.append(found_values)

        except ValueError as ve:
            print(f"ValueError for file {file_name}: {ve}")
        except Exception as e:
            print(f"An error occurred for file {file_name}: {e}")

# Create a new DataFrame from the found data
found_df = pd.DataFrame(found_data)

# Save the new DataFrame to an Excel file
output_file_path = os.path.join(output_dir, 'found_charges_grants_and_capitalgrants_with_filenames.xlsx')
found_df.to_excel(output_file_path, index=False)

print(f"Found values saved to {output_file_path}")


Found values saved to C:/Users/sarah/Downloads/ouput\found_charges_grants_and_capitalgrants_with_filenames.xlsx


In [22]:
import pandas as pd
import os

# Define the directory where the Excel files are saved
output_dir = 'C:/Users/sarah/Downloads/ouput'

# Keywords for searching and corresponding column names
keyword_dict = {
    'Total Government Activities': 'ga_chgs',
    'Total Business-Type Activities': 'bta_chgs',
    'Total Component Units': 'comp_chgs'
}

# Initialize a list to store the found values with their respective filenames
found_data = []

# Loop through each Excel file in the output directory
for file_name in os.listdir(output_dir):
    if file_name.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(output_dir, file_name)

        try:
            # Read the Excel file into a DataFrame, specifying the engine
            df = pd.read_excel(file_path, engine='openpyxl')

            # Initialize a dictionary for storing the values for this file, with 'file_name'
            found_values = {'file_name': file_name}
            for key in keyword_dict.values():
                found_values[key] = None  # Initialize each column as None

            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Charges for Services' in df.columns:
                for keyword, new_key in keyword_dict.items():
                    # Attempt to find the row that contains the keyword
                    row_mask = df['Functions/Programs'].str.contains(keyword, case=False, na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Charges for Services']
                        found_values[new_key] = value

            # Append the found values for this file (including file name) to the list
            found_data.append(found_values)

        except ValueError as ve:
            print(f"ValueError for file {file_name}: {ve}")
        except Exception as e:
            print(f"An error occurred for file {file_name}: {e}")

# Create a new DataFrame from the found data
found_df = pd.DataFrame(found_data)

# Save the new DataFrame to an Excel file
output_file_path = os.path.join(output_dir, 'found_charges_for_services_with_filenames.xlsx')
found_df.to_excel(output_file_path, index=False)

print(f"Found values saved to {output_file_path}")


Found values saved to C:/Users/sarah/Downloads/ouput\found_charges_for_services_with_filenames.xlsx


In [18]:
# Define the directory where the Excel files are saved
output_dir = 'C:/Users/sarah/Downloads/ouput'
keyword_dict = {
    'Total Government Activities': 'ga_chgs',
    'Total Business-Type Activities': 'bta_chgs',
    'Total Component Units': 'comp_chgs'
}

# Initialize a dictionary to store found values
found_values = {key: None for key in keyword_dict.values()}

# Loop through each Excel file in the output directory
for file_name in os.listdir(output_dir):
    if file_name.endswith('.xlsx'):
        # Construct the full file path
        file_path = os.path.join(output_dir, file_name)

        try:
            # Read the Excel file into a DataFrame, specifying the engine
            df = pd.read_excel(file_path, engine='openpyxl')

            # Check if the relevant columns are present in the DataFrame
            if 'Functions/Programs' in df.columns and 'Charges for Services' in df.columns:
                for keyword, new_key in keyword_dict.items():
                    # Attempt to find the row that contains the keyword
                    row_mask = df['Functions/Programs'].str.contains(keyword,case=False, na=False)
                    if row_mask.any():
                        row_index = df[row_mask].index[0]  # Get the first matching index
                        value = df.loc[row_index, 'Charges for Services']
                        found_values[new_key] = value

        except ValueError as ve:
            print(f"ValueError for file {file_name}: {ve}")
        except Exception as e:
            print(f"An error occurred for file {file_name}: {e}")

# Create a new DataFrame from the found values
found_df = pd.DataFrame([found_values])

# Save the new DataFrame to an Excel file
output_file_path = os.path.join(output_dir, 'found_charges_for_services.xlsx')
found_df.to_excel(output_file_path, index=False)

print(f"Found values saved to {output_file_path}")


Found values saved to C:/Users/sarah/Downloads/ouput\found_charges_for_services.xlsx


In [1]:
# Initialize a dictionary to store extracted values
extracted_values = {
    'ga_chgs': None,
    'bta_chgs': None,
    'comp_chgs': None
}

# Define the keywords for searching
keywords = {
    'ga_chgs': 'Total Government Activities',
    'bta_chgs': 'Total Business-Type Activities',
    'comp_chgs': 'Total Component Units'
}

# Iterate through the tables_dict to find the values
for key in tables_dict.keys():
    for i, df in enumerate(tables_dict[key]):
        # Check if the 'Charges for Services' column exists in the current DataFrame
        if 'Charges for Services' in df.columns:
            for new_key, keyword in keywords.items():
                # Attempt to find the row that contains the keyword
                row_mask = df.iloc[:, 0].str.contains(keyword, na=False)
                if row_mask.any():
                    row_index = df[row_mask].index[0]  # Get the first matching index
                    value = df.loc[row_index, 'Charges for Services']
                    extracted_values[new_key] = value
        else:
            print("not found")

# Create a new DataFrame from the extracted values
extracted_df = pd.DataFrame([extracted_values])

# Save the new DataFrame to an Excel file
output_file_path = 'C:/Users/sarah/Downloads/extracted_charges_for_services.xlsx'  # Specify your desired output file path
extracted_df.to_excel(output_file_path, index=False)

print(f"Extracted values saved to {output_file_path}")


NameError: name 'tables_dict' is not defined

In [7]:
# Define a function to rename columns based on index
def rename_columns(df):
    # Define a dictionary mapping the index of the column to the desired name
    rename_map = {
        0: 'Functions/Programs',
        2: 'Expenses',
        4: 'Charges for Services',         # 3rd column
        6: 'Operating Grants and Contributions',  # 5th column
        8: 'Capital Grants and Contributions',    # 7th column
        10: 'Net'                           # 9th column (only if there are 9 or more columns)
    }
    
    # Get the current number of columns
    num_columns = len(df.columns)
    
    # Loop through the rename_map and rename only if the column index exists in the DataFrame
    for index, new_name in rename_map.items():
        if index < num_columns:
            df.columns.values[index] = new_name


In [8]:
# # Iterate through the DataFrames in tables_dict and rename columns
# for key in tables_dict.keys():
#     print(f"\nRenaming columns for DataFrames under: {key}")
    
#     # Iterate through the DataFrames for the current key
#     for i, df in enumerate(tables_dict[key]):
#         # Print DataFrame column names before renaming
#         print(f"\nBefore renaming: DataFrame {i + 1} for {key}")
#         print("Columns before renaming:", df.columns)  # Print column names instead of entire df
        
#         # Rename the columns using the custom function
#         rename_columns(df)
        
#         # Print DataFrame column names after renaming
#         print(f"\nAfter renaming: DataFrame {i + 1}")
#         print("Columns after renaming:", df.columns)  # Print renamed column names



Renaming columns for DataFrames under: C:/Users/sarah/Downloads/OR2023ACFR.pdf

Before renaming: DataFrame 1 for C:/Users/sarah/Downloads/OR2023ACFR.pdf
Columns before renaming: Index(['Functions/Programs', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2',
       'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7',
       'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')

After renaming: DataFrame 1
Columns after renaming: Index(['Functions/Programs', 'Unnamed: 0', 'Charges for Services',
       'Unnamed: 2', 'Operating Grants and Contributions', 'Unnamed: 4',
       'Capital Grants and Contributions', 'Unnamed: 6', 'Net', 'Unnamed: 8',
       'Unnamed: 9'],
      dtype='object')

Before renaming: DataFrame 2 for C:/Users/sarah/Downloads/OR2023ACFR.pdf
Columns before renaming: Index(['Personal Income Taxes', 'Unnamed: 0', '13,641,111', 'Unnamed: 1', '-',
       'Unnamed: 2', '13,641,111.1', 'Unnamed: 3', '-.1'],
      dtype='object')

After renaming: DataFrame 2
Columns afte

In [37]:
for key in tables_dict.keys():
    print(tables_dict[key])
    for i in tables_dict[key]:
        #write code to look for the variable names


[                    Functions/Programs Unnamed: 0  Unnamed: 1 Unnamed: 2  \
0                  Primary Government:        NaN         NaN        NaN   
1             Governmental Activities:        NaN         NaN        NaN   
2                            Education          $   9,102,586          $   
3                       Human Services        NaN  22,495,019        NaN   
4                        Public Safety        NaN   2,368,376        NaN   
5   Economic and Community Development        NaN   1,240,288        NaN   
6                    Natural Resources        NaN   1,115,340        NaN   
7                       Transportation        NaN   2,173,145        NaN   
8       Consumer and Business Services        NaN    5 20,636        NaN   
9                       Administration        NaN    9 67,392        NaN   
10                         Legislative        NaN     7 6,896        NaN   
11                            Judicial        NaN    6 38,076        NaN   
12         

In [34]:
df[0]

,Unnamed: 0,Unnamed: 1,2023,Unnamed: 2,2022*,Unnamed: 3,2023.1,Unnamed: 4,2022*.1,Unnamed: 5,2023.2,Unnamed: 6,2022*.2
0,Current and other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,noncurrent assets,$,"45,554.7",$,"42,970.1",$,"2,902.6",$,"1,774.2",$,"48,457.3",$,"44,744.3"
2,"Capital assets, net",NaN,"32,199.0",NaN,"31,785.2",NaN,0.3,NaN,0.6,NaN,"32,199.3",NaN,"31,785.8"
3,Total Assets,NaN,"77,753.7",NaN,"74,755.3",NaN,"2,902.9",NaN,"1,774.8",NaN,"80,656.6",NaN,"76,530.1"
4,Deferred Outflows of Resources,NaN,"44,023.8",NaN,"45,227.5",NaN,-,NaN,-,NaN,"44,023.8",NaN,"45,227.5"
5,Current liabilities,NaN,"15,816.7",NaN,"15,323.0",NaN,634.3,NaN,573.7,NaN,"16,451.0",NaN,"15,896.7"
6,Noncurrent liabilities,NaN,"196,724.1",NaN,"208,522.2",NaN,173.8,NaN,186.5,NaN,"196,897.9",NaN,"208,708.7"
7,Total Liabilities,NaN,"212,540.8",NaN,"223,845.2",NaN,808.1,NaN,760.2,NaN,"213,348.9",NaN,"224,605.4"
8,Deferred Inflows of Resources,NaN,"79,298.0",NaN,"77,523.4",NaN,-,NaN,-,NaN,"79,298.0",NaN,"77,523.4"
9,Net Position:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
tables_dict['C:/Users/sarah/Downloads/OR2023ACFR.pdf']

[      Capital Assets (net)  17,385,072     635,019  18,020,091   7,708,411
 0  Total Noncurrent Assets  24,841,150   3,059,997  27,901,147  15,692,991
 1             Total Assets  48,954,089  10,745,046  59,699,135  23,286,027,
                                      Unnamed: 0 Unnamed: 1 Governmental  \
 0                                           NaN        NaN   Activities   
 1                                          SETS        NaN          NaN   
 2                                 rrent Assets:        NaN          NaN   
 3                     Cash and Cash Equivalents          $   15,383,971   
 4        Cash and Cash Equivalents - Restricted        NaN    2,731,702   
 5                                   Investments        NaN    1,499,544   
 6                      Investments - Restricted        NaN      484,119   
 7                 Securities Lending Collateral        NaN       11,634   
 8        Accounts and Interest Receivable (net)        NaN    1,923,405   
 9         

## Table 3 - Statement of Revenues, Expenditures, and changes in fund balance

### Finding page numbers

In [51]:
def find_keywords_in_pdf(keywords, pdf_file):
    pg=[]
    # Open the PDF file
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        # Loop through all pages
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text().lower()
            
            # Check if all keywords exist on this page
            if all(keyword.lower() in text for keyword in keywords):
                if page_num>=32 and page_num<100:
                    pg.append(page_num+1)
                    break
                #print(f"Keywords '{keywords}' found on page {page_num + 1}")
    return pg[0] if pg else None



### Looping through each pdf

In [52]:
pdf_file = ['C:/Users/sarah/Downloads/OR2023ACFR.pdf','C:/Users/sarah/Downloads/SC2023ACFR.pdf', 'C:/Users/sarah/Downloads/MI2023ACFR.pdf', 'C:/Users/sarah/Downloads/KS2023ACFR.pdf', 'C:/Users/sarah/Downloads/NJ2023ACFR.pdf']
tables_dict={}
for pdf in pdf_file:
    first_page_number = find_keywords_in_pdf(
        ["STATEMENT OF REVENUES, EXPENDITURES, AND CHANGES IN FUND BALANCE", "General"], 
        pdf
    )
    tables=extract_tables_from_pdf(pdf,first_page_number)
    df_list = []
    for i, table in enumerate(tables):
        df = pd.DataFrame(table)
        df_list.append(df)
    tables_dict[pdf] = df_list
    print(first_page_number)

47
None
43


unknown widths : 
[0, IndirectObject(1752, 0, 1541625150880)]
unknown widths : 
[0, IndirectObject(1756, 0, 1541625150880)]
unknown widths : 
[0, IndirectObject(1760, 0, 1541625150880)]
unknown widths : 
[0, IndirectObject(1764, 0, 1541625150880)]
unknown widths : 
[0, IndirectObject(1768, 0, 1541625150880)]
unknown widths : 
[0, IndirectObject(1772, 0, 1541625150880)]


40
48


## Table 4 - Balance sheet

### Finding page numbers

In [22]:
def find_keywords_in_pdf(keywords, pdf_file):
    pg=[]
    # Open the PDF file
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        # Loop through all pages
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text().lower()
            
            # Check if all keywords exist on this page
            if page_num>=32 and page_num<100:
                    pg.append(page_num+1)
                    break

# Example usage
#find_keywords_in_pdf(["BALANCE SHEET", "General","Federal"], "C:/Users/sarah/Downloads/OR2023ACFR.pdf")


Keywords '['BALANCE SHEET', 'General', 'Federal']' found on page 43
Keywords '['BALANCE SHEET', 'General', 'Federal']' found on page 209


### Looping through each pdf

In [53]:
pdf_file = ['C:/Users/sarah/Downloads/OR2023ACFR.pdf','C:/Users/sarah/Downloads/SC2023ACFR.pdf', 'C:/Users/sarah/Downloads/MI2023ACFR.pdf', 'C:/Users/sarah/Downloads/KS2023ACFR.pdf', 'C:/Users/sarah/Downloads/NJ2023ACFR.pdf']
tables_dict={}
for pdf in pdf_file:
    first_page_number = find_keywords_in_pdf(
        ["BALANCE SHEET", "General","Federal"], 
        pdf
    )
    tables=extract_tables_from_pdf(pdf,first_page_number)
    df_list = []
    for i, table in enumerate(tables):
        df = pd.DataFrame(table)
        df_list.append(df)
    tables_dict[pdf] = df_list
    print(first_page_number)

43
42
70


unknown widths : 
[0, IndirectObject(1752, 0, 1541581297936)]
unknown widths : 
[0, IndirectObject(1756, 0, 1541581297936)]
unknown widths : 
[0, IndirectObject(1760, 0, 1541581297936)]
unknown widths : 
[0, IndirectObject(1764, 0, 1541581297936)]
unknown widths : 
[0, IndirectObject(1768, 0, 1541581297936)]
unknown widths : 
[0, IndirectObject(1772, 0, 1541581297936)]


97
46


## Table 5 - Statement of Net Positions, Proprietary Funds

### Finding page numbers

In [18]:
def find_keywords_in_pdf(keywords, pdf_file):
    pg=[]
    # Open the PDF file
    with open(pdf_file, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        
        # Loop through all pages
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            text = page.extract_text().lower()
            
            # Check if all keywords exist on this page
            if page_num>=32 and page_num<100:
                    pg.append(page_num+1)
                    break

# Example usage
#find_keywords_in_pdf(["Statement of Net Position","PROPRIETARY FUNDS", "Total"], "C:/Users/sarah/Downloads/SC2023ACFR.pdf")

Keywords '['Statement of Net Position', 'PROPRIETARY FUNDS', 'Total']' found on page 50
Keywords '['Statement of Net Position', 'PROPRIETARY FUNDS', 'Total']' found on page 52


### Looping through each pdf

In [ ]:
pdf_file = ['C:/Users/sarah/Downloads/OR2023ACFR.pdf','C:/Users/sarah/Downloads/SC2023ACFR.pdf', 'C:/Users/sarah/Downloads/MI2023ACFR.pdf', 'C:/Users/sarah/Downloads/KS2023ACFR.pdf', 'C:/Users/sarah/Downloads/NJ2023ACFR.pdf']
tables_dict={}
for pdf in pdf_file:
    first_page_number = find_keywords_in_pdf(
        ["BALANCE SHEET", "General","Federal"], 
        pdf
    )
    tables=extract_tables_from_pdf(pdf,first_page_number)
    df_list = []
    for i, table in enumerate(tables):
        df = pd.DataFrame(table)
        df_list.append(df)
    tables_dict[pdf] = df_list
    print(first_page_number)